<a href="https://colab.research.google.com/github/SarveshPatil99/Adversarial-Robustness-Enhancement/blob/main/TML_Adversarial_Test_Set_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

select_a_dataset = "StyleGAN3 10000" # @param ["StyleGAN3 5000", "StyleGAN3 10000", "StyleGAN2 5000"]

if select_a_dataset == "StyleGAN3 5000":
  !gdown https://drive.google.com/uc?id=1-BTtMGzH8925FtIu20KbkfmmSz3LNTer
  !unzip -q original_dataset.zip
elif select_a_dataset == "StyleGAN3 10000":
  !gdown https://drive.google.com/uc?id=18MZZwgKjTGlB2Y4Esw_lRIvbpOaANbnh
  !unzip -q original_dataset_stylegan3_10000.zip
elif select_a_dataset == "StyleGAN2 5000":
  !gdown https://drive.google.com/uc?id=1ZBxnHMArN2-HAdViUFKarKxuXVk4sREk
  !unzip -q original_dataset_stylegan2_5000.zip

Downloading...
From: https://drive.google.com/uc?id=18MZZwgKjTGlB2Y4Esw_lRIvbpOaANbnh
To: /content/original_dataset_stylegan3_10000.zip
100% 2.12G/2.12G [00:19<00:00, 110MB/s]


In [ ]:
import os

# Define the adversarial attack types
adversarial_attacks = ['fgsm', 'bim', 'pgd']

# Directory structure for the test phase
dir_structure = {
    'test': ['real', 'fake']
}

# Create base directories for each adversarial attack type
for attack in adversarial_attacks:
    # Create a base directory for the current attack (e.g., 'fgsm_test')
    base_attack_dir = f'adversarial_{attack}'

    for phase in dir_structure:
        # Create a sub-directory for test (e.g., 'fgsm_test/test')
        phase_dir = os.path.join(base_attack_dir, phase)

        for category in dir_structure[phase]:
            # Create sub-directories for real and fake categories (e.g., 'fgsm_test/test/real')
            category_dir = os.path.join(phase_dir, category)
            os.makedirs(category_dir, exist_ok=True)
            print(f"Created directory: {category_dir}")

Created directory: adversarial_fgsm/test/real
Created directory: adversarial_fgsm/test/fake
Created directory: adversarial_bim/test/real
Created directory: adversarial_bim/test/fake
Created directory: adversarial_pgd/test/real
Created directory: adversarial_pgd/test/fake


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('/content/baseline_sg3=20k_n=8_lr=1e-03_val91.h5')

In [ ]:
!pip install adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 52.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.13.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.3 which is incompatible.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Prepare the data generator for the test set
test_datagen = ImageDataGenerator(
    rescale=1./255
    # Add any necessary preprocessing here
)

# ART classifier wrapper
art_classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,
    input_shape=(256, 256, 3),
    loss_object=tf.keras.losses.CategoricalCrossentropy(),
    clip_values=(0, 1)
)

# Create FGSM attack
attack = FastGradientMethod(estimator=art_classifier, eps=0.25)

# Directory where the adversarial images will be stored
adv_dir = 'adversarial_fgsm/test'

# Create the directories if they do not exist
os.makedirs(os.path.join(adv_dir, 'real'), exist_ok=True)
os.makedirs(os.path.join(adv_dir, 'fake'), exist_ok=True)

# Function to load images, generate adversarial examples and save them
def generate_and_save_adversarial(class_dir):
    # Loop through each file in the class directory
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        img = image.load_img(img_path, target_size=(256, 256))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img /= 255.  # Rescale pixel values

        # Generate adversarial image
        img_adv = attack.generate(x=img)

        # Determine whether it's real or fake for directory purposes
        if 'real' in class_dir:
            save_dir = os.path.join(adv_dir, 'real')
        else:
            save_dir = os.path.join(adv_dir, 'fake')

        # Save the adversarial image
        adv_img_path = os.path.join(save_dir, img_file)
        image.array_to_img(img_adv[0]).save(adv_img_path)

# Generate and save adversarial images for 'real' class
real_dir = os.path.join('original', 'test', 'real')
generate_and_save_adversarial(real_dir)

# Generate and save adversarial images for 'fake' class
fake_dir = os.path.join('original', 'test', 'fake')
generate_and_save_adversarial(fake_dir)

print("Adversarial images generated and stored.")

Adversarial images generated and stored.


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

# Load the trained model
model = load_model('/content/baseline_sg3=20k_n=8_lr=1e-03_val91.h5')

# Define the path to the adversarial test set
adv_test_dir = 'adversarial_fgsm/test'

# Prepare the data generator for the adversarial test set
adv_test_datagen = ImageDataGenerator(rescale=1./255)

# Since we're using the data generator, we need the classes to be ordered as they are in the 'class_indices'
# This might differ based on how your original 'test' generator was created.
classes = ['real', 'fake']

# Create the generator for the adversarial test set
adv_test_generator = adv_test_datagen.flow_from_directory(
    directory=adv_test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,  # Important for maintaining label order
    classes=classes
)

# Predict on the adversarial test set
adv_predictions = model.predict(adv_test_generator, verbose=1)
adv_predicted_classes = np.argmax(adv_predictions, axis=1)

# Get the true labels from the generator
adv_true_labels = adv_test_generator.classes

# Calculate the accuracy
adv_accuracy = accuracy_score(adv_true_labels, adv_predicted_classes)
print(f'Accuracy on adversarial test set: {adv_accuracy:.2f}')

Found 2000 images belonging to 2 classes.
63/63 [==============================] - 9s 138ms/step
Accuracy on adversarial test set: 0.54


In [ ]:
!zip -r adversarial_fgsm.zip adversarial_fgsm

  adding: adversarial_fgsm/ (stored 0%)
  adding: adversarial_fgsm/test/ (stored 0%)
  adding: adversarial_fgsm/test/real/ (stored 0%)
  adding: adversarial_fgsm/test/real/09675.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09665.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09720.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09923.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09990.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09392.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09133.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09737.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09140.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09352.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09814.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09953.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09425.png (deflated 0%)
  adding: adversarial_fgsm/test/real/09426.png (deflated 0%)
  adding: a

In [ ]:
!cp /content/adversarial_fgsm.zip /content/drive/MyDrive/TML_Project/data/